In [1]:
import pandas as pd
import psycopg2
import json
import pathlib
import regex
import re

In [2]:
data_dir = "../data"
data_folders = [d for d in pathlib.Path(data_dir).iterdir() if d.is_dir()]

In [3]:
data_folders

[PosixPath('../data/stock_prices'),
 PosixPath('../data/symbols'),
 PosixPath('../data/profiles')]

In [118]:
conn.close()

In [119]:
# conn = psycopg2.connect("host=localhost dbname=financialmodelling user=financialmodelling_system password=system")
conn = psycopg2.connect("host=localhost dbname=financialmodelling user=postgres password=mysecretpassword")
cursor = conn.cursor()

In [120]:
# Upload symbols data first
profiles = [d for d in data_folders if "profiles" in d.stem][0]
profiles_files = [fp for fp in profiles.iterdir()]

In [121]:
for fp in profiles_files:
    with open(fp, "r") as f:
        data = json.load(f)
        values = {
            "symbol_code": data.get("symbol", ""),
            "company_name": data["profile"].get("companyName", ""),
            "exchange_name": data["profile"].get("exchange", ""),
            "industry_name": data["profile"].get("industry", ""),
            "website_link": data["profile"].get("website", ""),
            "company_description": data["profile"].get("description", ""),
            "ceo_name": data["profile"].get("ceo", ""),
            "sector_name": data["profile"].get("sector", ""),
            "company_image": data["profile"].get("image", ""),
        }
        values = {
            k: v for k, v in values.items() if v is not None
        }
        parsed_values = [
            (col, val.replace("'", "''")) for col, val in values.items() if len(val) > 0
        ]
        cols = ",\n\t".join([c[0] for c in parsed_values])
        vals = ",\n\t".join([f"'{c[1]}'" for c in parsed_values])
        sql_template = f"""
        INSERT INTO financialmodelling.dim_symbol (
            {cols}
        )
        VALUES (
            {vals}
        )
        """
        cursor.execute(sql_template)
        conn.commit()

In [117]:
data

{'symbol': 'ERYP.PA',
 'profile': {'price': 6.41,
  'beta': '1.556717',
  'volAvg': '59981',
  'mktCap': '1.16144712E8',
  'lastDiv': None,
  'range': '3.4-8.94',
  'changes': 0.43,
  'changesPercentage': '(+7.19%)',
  'companyName': 'ERYTECH Pharma S.A.',
  'exchange': 'Paris',
  'industry': 'Biotechnology',
  'website': 'http://www.erytech.com',
  'description': "ERYTECH Pharma S.A., a clinical-stage biopharmaceutical company, develops red blood cell-based therapeutics for severe forms of cancer and orphan diseases. The company's lead product candidate is eryaspase, which is in Phase III clinical development for the treatment of second-line pancreatic cancer and in Phase II stage for the treatment of triple-negative breast cancer. ERYTECH Pharma S.A. has research collaboration with the Fox Chase Cancer Center to advance the preclinical development of erymethionase for the treatment of homocystinuria; and Queen's University of Canada to advance the preclinical development of eryminase